# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    20-Jan-2026**

**News Source / Catalyst:** Indonesia’s rupiah falls as Prabowo nominates nephew to central bank role


**Markets (s) Impacted:** FX (ASEAN), FI (Local Rates/Curves), Sovereign Credit

**Trade Taxonomy:** Macro RV / Institutional Divergence / Governance Premium

**Trade Idea (1 sentence):**
> Long SGD & MYR vs Short IDR to monetise widening institutional credibility spreads as Indonesia drifts toward growth-driven fiscal dominance, while Singapore & Malaysia retain monetary orthodoxy

**Client/Desk Pitch (30 seconds):**
> Indonesia’s growth target shift (5% → 8%) requires coordination between fiscal and monetary arms, and markets are now pricing the risk of policy subordination. IDR is near post-crisis lows as BI’s independence is questioned and fiscal deficits push against legal limits. In contrast, Singapore and Malaysia remain institutionally orthodox with cleaner governance signals and improving balance sheet credibility. We recommend Long SGD & MYR vs Short IDR as a USD-neutral ASEAN institutional spread that monetises credibility differentials without importing US macro noise.

**Why Now:**
- IDR testing 1998 Asian Crisis levels
- Parliament reviewing mandate expansions & removal powers
- Growth target raised to 8% without structural capacity
- Fiscal deficit at 2.92% vs 3% legal cap
- investor sentiment turning on BI independence

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- Indonesia → entering a fiscal dominance regime (growth > stability)
- Singapore → policy orthodoxy & capital inflow magnet
- Malaysia → external surplus + manageable fiscal trajectory

**Long Leg (Tickers + Rationale):**
- SGD (SGDUSD=X) → NEER targeting, policy orthodoxy, safe haven within ASEAN
- MYR (MYRUSD=X) → moderate carry & improving trade composition

**Short Leg (Tickers + Rationale):**
- IDR (IDRUSD=X) → central bank capture risk + narrowing real yields + deficit expansion

**Weights:** equal-weight (50% SGD , 50% MYR vs 100% IDR)

**Trade Structure:**

To avoid USD contamination: Basket = 0.5 * SGDINR + 0.5 * MYRIDR


---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |       W           |   USD-neutral construction    |
| Oil Beta             |       M           |   MYR/IDR commodity channel    |
| Rates Duration       |       M           |    BI may be forced to ease   |
| Governance Risk      |       S           |  Core catalyst     |
| Risk-On/Off          |       M           |  SGD benefits in risk-off     |
| China/EM Beta        |       W           |  Domestic politics drive IDR     |
| Volatility Regime    |       M           |  Political vol > macro vol     |

**Catalyst Type:** Institutional Credibility / Governance

**Realisation Window:** 4 - 8 weeks. 

**Holding Period Selected:**  45 trading days

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** (units per leg)  5% notional (2.5% per long leg), lever-neutral

**Stop-Loss:** -2.2% basket level

**Take-Profit (optional):**  5.5%

**Re-entry:** Yes if USD-driven selloffs contaminate IDR  

**FX-Hedging Required:** None (deliberate USD-neutral)

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [41]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [42]:
# --- Parameters ---

long_fx = ["SGDUSD=X", "MYRUSD=X"]    # SGD, MYR
short_fx = ["IDRUSD=X"]               # IDR (risk leg)

start_date = "2015-01-01"
holding_days = 45

stop_loss = -0.022      # -2.2%
take_profit = 0.055      # +5.5%

weights_long = np.array([0.5, 0.5])
weights_short = np.array([1.0])  # IDR only


In [43]:
# --- Data Download ---

tickers = long_fx + short_fx
data = yf.download(tickers, start=start_date)["Close"].dropna().ffill()

# --- Compute log returns ---
log_rets = np.log(data / data.shift(1)).dropna()

# --- Building Crosses ---

crosses = pd.DataFrame(index=data.index)

crosses["SGDIDR"] = data["SGDUSD=X"] / data["IDRUSD=X"]
crosses["MYRIDR"] = data["MYRUSD=X"] / data["IDRUSD=X"]

cross_log_rets = np.log(crosses / crosses.shift(1)).dropna()



/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_20899/4003900664.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"].dropna().ffill()
[*********************100%***********************]  3 of 3 completed
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [44]:
# --- Spread Returns ---

long_basket = (cross_log_rets.mul(weights_long, axis=1)).sum(axis=1)

strategy_rets = long_basket

In [ ]:
# --- BACKTEST LOOP ---

trade_dates = []
trade_pnl = []

dates = strategy_rets.index
n = len(strategy_rets)
i = 0

while i < n - holding_days:
    entry = dates[i]
    pnl = 0.0
    stopped = False
    
    for j in range(holding_days):
        daily = strategy_rets.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily) - 1
        
        if pnl <= stop_loss:
            trade_dates.append(entry)
            trade_pnl.append(pnl)
            stopped = True
            break
        
        if pnl >= take_profit:
            trade_dates.append(entry)
            trade_pnl.append(pnl)
            stopped = True
            break
    
    if not stopped:
        trade_dates.append(entry)
        trade_pnl.append(pnl)
    
    i += holding_days

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})


In [46]:
# --- Performance Summary ---

summary = pd.Series({
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
})

print("\n=== STRATEGY SUMMARY ===\n")
print(summary.round(4))





=== STRATEGY SUMMARY ===

Total Trades         63.0000
Win Rate              0.4762
Average PnL           0.0009
Best Trade            0.0666
Worst Trade          -0.0721
Stop-Out Frequency    0.2698
dtype: float64


## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 
- Win Rate:
- Avg PnL:
- Best:
- Worst: 
- Stop-Outs: 

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [47]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [48]:
# --- Computing Synthetic Basket Price Series ---

def trade_id(prefix="RV"):
    return f"{prefix}-{np.random.randint(1e8):08d}"

blotter_rows = []
i = 0

while i < n - holding_days:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_days - 1, n - 1)]

    entry_price = long_basket.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    status = "CLOSED"

    for j in range(holding_days):
        daily = strategy_rets.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            status = "STOPPED"
            stopped = True
            break
        
        if pnl >= take_profit:
            exit_date = dates[i + j]
            status = "TAKE_PROFIT"
            stopped = True
            break

    exit_price = long_basket.loc[exit_date]
    holding = (exit_date - entry_date).days

    blotter_rows.append({
        "TradeID": trade_id("RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "ASEAN Governance Spread",
        "LongLeg": "0.5 SGDIDR + 0.5 MYRIDR",
        "ShortLeg": "IDR",
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100,
        "Status": status,
        "HoldingPeriod": holding,
        "Notes": ""
    })
    
    i += holding_days

blotter = pd.DataFrame(blotter_rows)
print("\n=== TRADE BLOTTER (LAST 5) ===\n")
print(blotter.tail())

# Export
blotter.to_csv("asean_governance_spread_blotter.csv", index=False)
trade_results.to_csv("asean_governance_spread_pnl.csv", index=False)


=== TRADE BLOTTER (LAST 5) ===

        TradeID            Timestamp                 Strategy  \
58  RV-57417489  2026-01-20 09:24:54  ASEAN Governance Spread   
59  RV-00173275  2026-01-20 09:24:54  ASEAN Governance Spread   
60  RV-44914435  2026-01-20 09:24:54  ASEAN Governance Spread   
61  RV-39460815  2026-01-20 09:24:54  ASEAN Governance Spread   
62  RV-04255186  2026-01-20 09:24:54  ASEAN Governance Spread   

                    LongLeg ShortLeg   EntryDate    ExitDate  EntryPrice  \
58  0.5 SGDIDR + 0.5 MYRIDR      IDR  2025-01-10  2025-03-13      0.0030   
59  0.5 SGDIDR + 0.5 MYRIDR      IDR  2025-03-14  2025-05-19     -0.0049   
60  0.5 SGDIDR + 0.5 MYRIDR      IDR  2025-05-20  2025-06-12     -0.0018   
61  0.5 SGDIDR + 0.5 MYRIDR      IDR  2025-07-22  2025-09-22      0.0030   
62  0.5 SGDIDR + 0.5 MYRIDR      IDR  2025-09-23  2025-10-28     -0.0010   

    ExitPrice  GrossPnL(%)  StopLoss(%)  TakeProfit(%)   Status  \
58    -0.0103       2.8239      -2.2000         5.50